In [5]:
from selenium import webdriver as wd
from selenium.webdriver.common.by import By
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from bs4 import BeautifulSoup as bs
from urllib.request import urlopen
from urllib import parse
import time, requests, datetime, re, json, os
import pandas as pd
from metadata import *

keyword = input("검색어: ")
num = int(input("스크롤 횟수: "))


driver = wd.Chrome(driver_path)
print(f"login start - option {login_option}")
login_url = "https://www.instagram.com/accounts/login/"
driver.get(login_url)
time.sleep(5)

is_login_success = False
if login_option == "instagram":
    try:
        instagram_id_form = driver.find_element(By.XPATH, instagram_id_name)
        instagram_id_form.send_keys(user_id)
        time.sleep(3)

        instagram_pw_form = driver.find_element(By.XPATH, instagram_pw_name)
        instagram_pw_form.send_keys(user_pw)
        time.sleep(3)
        
        login_ok_button =  driver.find_element(By.XPATH, instagram_login_btn)
        login_ok_button.click()
        is_login_success = True
    except:
        print("instagram login fail")
        is_login_success = False
    time.sleep(10)


if is_login_success:
    print(f"login {login_option} success")
    print(f"Start {keyword} Extract")
    keyword = parse.quote(keyword)
    url = f"https://www.instagram.com/explore/tags/{keyword}/"
    driver.get(url)
    time.sleep(5)
    
## 수정 필요
    # 스크롤 다운
    before_location = driver.execute_script("return window.pageYOffset")
    scroll_location = driver.execute_script("return document.body.scrollHeight")
    while True:
        #현재 위치 + 지정 수 만큼 스크롤 이동
        driver.execute_script("window.scrollTo(0,{})".format(scroll_location + num))
            
        #전체 스크롤이 늘어날 때까지 대기
        time.sleep(2)
        
        #이동 후 스크롤 위치
        after_location = driver.execute_script("return window.pageYOffset")
        
        #이동후 위치와 이동 후 위치가 같으면(더 이상 스크롤이 늘어나지 않으면) 종료
        if before_location == after_location:
            break

        #같지 않으면 다음 조건 실행
        else:
            #이동여부 판단 기준이 되는 이전 위치 값 수정
            before_location = driver.execute_script("return window.pageYOffset")

# 본문 url
html = driver.page_source
soup = bs(html, 'html.parser')
soup2 = soup.find("article", class_="_aao7")
soup3 = soup2.find_all("div", class_="_aabd _aa8k _aanf")
print(len(soup3),"건")
for i in soup3:
    driver.get("https://www.instagram.com/"+i.a['href'])
    time.sleep(2)
    html = driver.page_source
    main_soup = bs(html, 'html.parser')
    
    # 태그
    main_soup_tag = main_soup.find_all("div", class_="_a9zs" )
    try:
        tags = re.findall('#[A-Za-z0-9가-힣]+', str(main_soup_tag[0]))
    except:
        tags = re.findall('#[A-Za-z0-9가-힣]+', str(main_soup_tag[1]))
    tag_data = "".join(tags)
    tag.append(tag_data.replace("#"," "))
    
    # 업로더
    main_soup_id = main_soup.find_all("div", class_="_aaqt")
    upload_id.append(main_soup_id[0].text)
    
    # 이미지
    img_src = main_soup.find_all("img", class_="_aagt")
    img_url.append(img_src[0].attrs['src'])

driver.close()


login start - option instagram
login instagram success
Start 네일아트 Extract
57 건


In [11]:
len(upload_id),len(tag),len(img_url)

(57, 57, 57)

In [10]:
insta_crawling_df = pd.DataFrame({"upload_id": upload_id,
                                                        "tag": tag,
                                                        "img_url": img_url})

In [12]:
insta_crawling_df

,upload_id,tag,img_url
0,banara9,,https://scontent-ssn1-1.cdninstagram.com/v/t51...
1,by_jood_,,https://scontent-ssn1-1.cdninstagram.com/v/t51...
2,nail_celine__,네일셀린 네일아트 이달의아트 이달의네일 화려한네일 예쁜네일 웨딩네일 웨딩네일아트 ...,https://scontent-ssn1-1.cdninstagram.com/v/t51...
3,ryun_jjang,륜쌀롱,https://scontent-ssn1-1.cdninstagram.com/v/t51...
4,ddung_nail__,,https://scontent-ssn1-1.cdninstagram.com/v/t51...
5,nailbareum,호박패디 마산네일 양덕동네일,https://scontent-ssn1-1.cdninstagram.com/v/t51...
6,_nail__eun,,https://scontent-ssn1-1.cdninstagram.com/v/t51...
7,nail_prescription,네일처방 네일고민해결 파츠네일 스와네일 네일재료 네일아트 컬쿤 컬처쿤스트,https://scontent-ssn1-1.cdninstagram.com/v/t51...
8,on_nail.yeonji,,https://scontent-ssn1-1.cdninstagram.com/v/t51...
9,riiiiiiiko_nail,nail nails jelnail nailstagram,https://scontent-ssn1-1.cdninstagram.com/v/t51...


In [ ]:
# 저장
# insta_crawling_df.to_csv("파일명", index=False)

xpath나 css는 새로고침하면 값이 바뀌어서 못씀